In [1]:
%load_ext autoreload

import os, sys
import networkx as nx

sys.path.insert(0, "../")
sys.path.insert(0, "/home/j/joppich/.local/lib/python3.11/site-packages/")
#otherwise leidenalg is used in an outdated version ...

import leidenalg as la
print("la", la.version)

%autoreload 2
from PICASO.kgraph import *
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from collections import defaultdict, Counter

import random
random.seed(42)

import pickle

la 0.10.1


In [2]:
kg = KGraph()
kg.load_kgraph("../data/initial_base_graph.out")

In [3]:
exprDF = pd.read_csv("./expression_ctcond_mean_df.tsv", sep="\t")

In [4]:
exprDFs = {}

for group in exprDF.group.unique():
    exprDFs[group] = exprDF[exprDF.group == group].copy()

print([x for x in exprDFs])

['ATL_AKI', 'ATL_COV_AKI', 'ATL_DKD', 'ATL_H_CKD', 'ATL_Ref', 'CNT_AKI', 'CNT_COV_AKI', 'CNT_DKD', 'CNT_H_CKD', 'CNT_Ref', 'DCT_AKI', 'DCT_COV_AKI', 'DCT_DKD', 'DCT_H_CKD', 'DCT_Ref', 'DTL_AKI', 'DTL_COV_AKI', 'DTL_DKD', 'DTL_H_CKD', 'DTL_Ref', 'EC_AKI', 'EC_COV_AKI', 'EC_DKD', 'EC_H_CKD', 'EC_Ref', 'FIB_AKI', 'FIB_COV_AKI', 'FIB_DKD', 'FIB_H_CKD', 'FIB_Ref', 'IC_AKI', 'IC_COV_AKI', 'IC_DKD', 'IC_H_CKD', 'IC_Ref', 'IMM_AKI', 'IMM_COV_AKI', 'IMM_DKD', 'IMM_H_CKD', 'IMM_Ref', 'NEU_AKI', 'NEU_COV_AKI', 'NEU_DKD', 'NEU_H_CKD', 'NEU_Ref', 'PC_AKI', 'PC_COV_AKI', 'PC_DKD', 'PC_H_CKD', 'PC_Ref', 'PEC_AKI', 'PEC_COV_AKI', 'PEC_DKD', 'PEC_H_CKD', 'PEC_Ref', 'POD_AKI', 'POD_COV_AKI', 'POD_DKD', 'POD_H_CKD', 'POD_Ref', 'PT_AKI', 'PT_COV_AKI', 'PT_DKD', 'PT_H_CKD', 'PT_Ref', 'PapE_AKI', 'PapE_COV_AKI', 'PapE_DKD', 'PapE_H_CKD', 'PapE_Ref', 'TAL_AKI', 'TAL_COV_AKI', 'TAL_DKD', 'TAL_H_CKD', 'TAL_Ref', 'VSM/P_AKI', 'VSM/P_COV_AKI', 'VSM/P_DKD', 'VSM/P_H_CKD', 'VSM/P_Ref']


In [5]:
exprKGs = {}

gsa = GenesetAnnotator()
mNS = MeanNetworkScorer()

for group in exprDFs:
    
    print(group)
    
    gKG = KGraph()
    
    gKG.load_kgraph("../data/initial_base_graph.out")

    gKG.add_gene_expression(exprDFs[group])
    mNS.score_nodes_from_properties(gKG)

    mNS.score(gKG)

    gsa.annotate_genesets(gKG, settype="disease", targettype="gene")
    gsa.annotate_genesets(gKG, settype="geneset", targettype="gene")
    gsa.annotate_genesets(gKG, settype="ncRNA", targettype="gene")
    gsa.annotate_genesets(gKG, settype="drug", targettype=["gene", "disease"])
            
    exprKGs[group] = gKG

ATL_AKI
Measured Genes 13035
Found Genes 12177


/mnt/raidbio/extproj/projekte/regulatory_networks/kpmp/../mikg/kgraph.py:1621: RuntimeWarning: invalid value encountered in scalar divide
  edge_zscore = (edge_score-etype2mean[etype])/etype2std[etype]


Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 target nodes for settype=drug and targettype=['gene', 'disease']
ATL_COV_AKI
Measured Genes 10860
Found Genes 10323
Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 target nodes for settype=drug and targettype=['gene', 'disease']
ATL_DKD
Measured Genes 7668
Found Genes 7276
Processed 7884 of 43646 target nodes for settype=disease and targettype=gene
Processed 42878 of 43646 target nodes for settype=geneset and targettype=gene
Processed 3390 of 43646 target nodes for settype=ncRNA and targettype=gene
Processed 3513 of 51488 target nodes

In [6]:
print(len(exprKGs))
import pickle

with open("ctcond_exprkgs.pickle", 'wb') as f:
    pickle.dump(exprKGs, f)

80
